In [11]:
from transformers import AutoTokenizer, AutoModel

In [12]:
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/chatglm/model/chatglm3-6b", trust_remote_code=True)
tokenizer

ChatGLMTokenizer(name_or_path='/root/autodl-tmp/chatglm/model/chatglm3-6b', vocab_size=64798, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=False)

In [13]:
import torch
model = AutoModel.from_pretrained("/root/autodl-tmp/chatglm/model/chatglm3-6b", 
                                  trust_remote_code=True,
                                  low_cpu_mem_usage=True,
                                  torch_dtype=torch.half, 
                                  device_map="auto")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
model.chat(tokenizer, "如何写简历？", history=[])

In [5]:
?model.chat

Signature:
model.chat(
    tokenizer,
    query: str,
    history: List[Dict] = None,
    role: str = 'user',
    max_length: int = 8192,
    num_beams=1,
    do_sample=True,
    top_p=0.8,
    temperature=0.8,
    logits_processor=None,
    **kwargs,
)
Docstring: <no docstring>
File:      ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b/modeling_chatglm.py
Type:      method

In [6]:
?tokenizer.build_chat_input

Signature: tokenizer.build_chat_input(query, history=None, role='user')
Docstring: <no docstring>
File:      ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b/tokenization_chatglm.py
Type:      method

In [7]:
tokenizer.build_chat_input("如何写简历？", history=[], role="user")

{'input_ids': tensor([[64790, 64792, 64795, 30910,    13, 51310, 55172, 36360, 31514, 64796]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'position_ids': tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])}

In [8]:
tokenizer.decode([64790, 64792, 64795, 30910,    13, 51310, 55172, 36360, 31514, 64796])

'[gMASK]sop<|user|> \n 如何写简历？<|assistant|>'

In [10]:
! cat  ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b/modeling_chatglm.py

""" PyTorch ChatGLM model. """

import math
import copy
import warnings
import re
import sys

import torch
import torch.utils.checkpoint
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, LayerNorm, MSELoss, BCEWithLogitsLoss
from torch.nn.utils import skip_init
from typing import Optional, Tuple, Union, List, Callable, Dict, Any
from copy import deepcopy

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
    CausalLMOutputWithPast,
    SequenceClassifierOutputWithPast,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import logging
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList, GenerationConfig, ModelOutput

from .configuration_chatglm import ChatGLMConfig

# flags required to enable jit fusion kernels

if sys.platform != 'darwin':
    torch._C._jit_set_profiling_mode(False)
    torch._

In [9]:
!  cat ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b/tokenization_chatglm.py

import json
import os
import re
from typing import List, Optional, Union, Dict
from sentencepiece import SentencePieceProcessor
from transformers import PreTrainedTokenizer
from transformers.utils import logging, PaddingStrategy
from transformers.tokenization_utils_base import EncodedInput, BatchEncoding


logger = logging.get_logger(__name__)


class SPTokenizer:
    def __init__(self, model_path: str):
        # reload tokenizer
        assert os.path.isfile(model_path), model_path
        self.sp_model = SentencePieceProcessor(model_file=model_path)

        # BOS / EOS token IDs
        self.n_words: int = self.sp_model.vocab_size()
        self.bos_id: int = self.sp_model.bos_id()
        self.eos_id: int = self.sp_model.eos_id()
        self.pad_id: int = self.sp_model.unk_id()
        assert self.sp_model.vocab_size() == self.sp_model.get_piece_size()

        role_special_tokens = ["<|system|>", "<|user|>", "<|assistant|>", "<|observation|>"]
        special_tokens = ["[MASK]",